# Backend API and Job System Demo

Demonstrates connecting to Promethium backend services via REST API.

**Prerequisites:** Running Promethium backend server

**Note:** This notebook requires the backend to be running. See deployment documentation.

In [ ]:
# !pip install promethium-seismic==1.0.0

In [ ]:
import promethium
import requests
import json
import time
import os

print(f"Promethium {promethium.__version__}")

# Backend configuration
BASE_URL = os.environ.get('PROMETHIUM_API_URL', 'http://localhost:8000')
API_PREFIX = '/api/v1'

print(f"Backend URL: {BASE_URL}")

## 1. Health Check

In [ ]:
def check_backend():
    """Check if backend is running."""
    try:
        r = requests.get(f"{BASE_URL}/health", timeout=5)
        if r.status_code == 200:
            print("Backend is running")
            return True
    except requests.exceptions.ConnectionError:
        print("Backend not available. Start with: docker-compose up")
    return False

backend_available = check_backend()

## 2. API Client

In [ ]:
class PromethiumClient:
    """Simple API client for Promethium backend."""
    
    def __init__(self, base_url=BASE_URL, prefix=API_PREFIX):
        self.base_url = base_url
        self.prefix = prefix
        
    def _url(self, endpoint):
        return f"{self.base_url}{self.prefix}{endpoint}"
    
    def list_datasets(self):
        r = requests.get(self._url('/datasets'))
        return r.json() if r.status_code == 200 else []
    
    def submit_job(self, dataset_id, job_type, config=None):
        payload = {
            'dataset_id': dataset_id,
            'job_type': job_type,
            'config': config or {}
        }
        r = requests.post(self._url('/jobs'), json=payload)
        return r.json() if r.status_code in (200, 201) else None
    
    def get_job_status(self, job_id):
        r = requests.get(self._url(f'/jobs/{job_id}'))
        return r.json() if r.status_code == 200 else None
    
    def wait_for_job(self, job_id, timeout=300, poll=5):
        start = time.time()
        while time.time() - start < timeout:
            status = self.get_job_status(job_id)
            if status and status.get('status') in ('completed', 'failed'):
                return status
            time.sleep(poll)
        return None

client = PromethiumClient()
print("Client initialized")

## 3. List Datasets

In [ ]:
if backend_available:
    datasets = client.list_datasets()
    print(f"Found {len(datasets)} datasets")
    for ds in datasets[:5]:
        print(f"  - {ds.get('name', 'unnamed')} (ID: {ds.get('id')})")
else:
    print("Backend not available - showing example flow")
    datasets = [{'id': 'demo-123', 'name': 'example_dataset'}]
    print(f"Example dataset: {datasets[0]}")

## 4. Submit Job

In [ ]:
# Example job configuration
job_config = {
    'model': 'unet_denoise_v1',
    'device': 'auto',
    'batch_size': 32
}

if backend_available and datasets:
    job = client.submit_job(datasets[0]['id'], 'reconstruction', job_config)
    if job:
        print(f"Job submitted: {job.get('id')}")
        print(f"Status: {job.get('status')}")
else:
    print("Example job submission:")
    print(f"  Dataset: demo-123")
    print(f"  Type: reconstruction")
    print(f"  Config: {job_config}")

## 5. Monitor Job

In [ ]:
# Job monitoring example
print("Job monitoring workflow:")
print("""
# Submit job
job = client.submit_job(dataset_id, 'reconstruction', config)

# Wait for completion
result = client.wait_for_job(job['id'], timeout=300)

# Check result
if result['status'] == 'completed':
    output_path = result['output_path']
    print(f"Results saved to: {output_path}")
""")

## Summary

This notebook demonstrated:
1. Backend health check
2. API client usage
3. Dataset listing
4. Job submission
5. Job monitoring

For deployment, see the Docker documentation.